In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [2]:
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_51259_2024-04-22~2025-04-22.xlsx')
ka_df = pd.read_excel('/Users/seanyu_mac/Downloads/20250422100214.xlsx')

In [3]:
fs_df['박스수량'] = fs_df['옵션명'].apply(lambda x: x.split('_')[1].split('박스')[0])
fs_df['박스수량'] = fs_df['박스수량'].astype(int)

In [4]:
fs_df['주문 상태'].unique()

array(['결제완료', '배송중', '배송완료'], dtype=object)

In [5]:
ka_df['박스수량'] = ka_df['옵션'].apply(lambda x: x.split('_')[1].split('박스')[0])
ka_df['박스수량'] = ka_df['박스수량'].astype(int)

In [6]:
fs_df = fs_df[['결제 번호', '주문 상태','결제 완료일', '옵션명', '수량', '박스수량', '정산가']]

In [7]:
ka_df = ka_df[['결제번호', '주문상태', '발송요청일', '옵션', '수량', '박스수량','정산기준금액']]

In [8]:
ka_df.columns = fs_df.columns

In [9]:
df = pd.concat([fs_df, ka_df])

In [10]:
df.columns = ['결제 번호', '주문 상태', '결제 완료일', '옵션명', '수량', '박스수량', '매출']

In [11]:
df['주문 상태'].unique()

array(['결제완료', '배송중', '배송완료', '304 배송 중', '204 결제 취소 완료', '305 배송 완료',
       '601 구매 결정'], dtype=object)

In [12]:
df[df['주문 상태'] == '204 결제 취소 완료']

,결제 번호,주문 상태,결제 완료일,옵션명,수량,박스수량,매출
15,2424751775,204 결제 취소 완료,2025-04-19 23:04:25,선택: 파이토레드큐민(정제)_3박스(25%할인),1,3,123750
26,2423770744,204 결제 취소 완료,2025-04-18 17:06:45,선택: 퓨어레드큐민(분말)_3박스(46%할인),1,3,79000
47,2422227224,204 결제 취소 완료,2025-04-16 19:25:27,선택: 퓨어레드큐민(분말)_3박스(46%할인),1,3,79000
72,2420835616,204 결제 취소 완료,2025-04-15 08:31:45,선택: 파이토레드큐민(정제)_3박스(25%할인),1,3,123750
86,2420062502,204 결제 취소 완료,2025-04-14 10:05:29,선택: 파이토레드큐민(정제)_12박스(45%할인),1,12,363000


In [13]:
df = df[df['주문 상태'].isin(['결제완료', '배송중', '배송완료', '202 배송 요청', '304 배송 중', '305 배송 완료', '601 구매 결정'])]

In [14]:
df['주문 상태'].unique()

array(['결제완료', '배송중', '배송완료', '304 배송 중', '305 배송 완료', '601 구매 결정'],
      dtype=object)

In [15]:
df['결제 완료일'] = pd.to_datetime(df['결제 완료일'])
df['결제 완료일'] = df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))
pivot_df = pd.pivot_table(df, index=['결제 완료일', '옵션명'], values=['매출', '박스수량'], aggfunc=['sum'], fill_value=0, margins=True, margins_name='총합')
pivot_df.columns = pivot_df.columns.droplevel(0)
temp_df = pd.DataFrame(pivot_df)

pivot = pd.pivot_table(df, index=['결제 완료일', '옵션명'], values=['결제 번호'], aggfunc='count', fill_value=0, margins=True, margins_name='총합')
temp_df2 = pd.DataFrame(pivot)

result_df = pd.merge(temp_df, temp_df2, left_index=True, right_index=True, how='inner')
result_df.rename(columns={'결제 번호': '주문수량'}, inplace=True)
result_df.to_excel(f'/Users/seanyu_mac/Downloads/샤인약사님_매출_{datetime.now()}.xlsx')
result_df

매출  박스수량  주문수량
결제 완료일     옵션명                                              
2025-04-14 선택: 파이토레드큐민(정제)_12박스(45%할인)    726000    24     2
           선택: 파이토레드큐민(정제)_3박스(25%할인)     371250     9     3
           선택: 파이토레드큐민(정제)_6박스(36%할인)     422400    12     2
           선택: 퓨어레드큐민(분말)_3박스(46%할인)      237000     9     3
           선택: 퓨어레드큐민(분말)_6박스(53%할인)      417000    12     2
           파이토레드큐민(정제)_12박스(45%할인)       1089000    36     3
           파이토레드큐민(정제)_3박스(25%할인)         123750     3     1
           파이토레드큐민(정제)_6박스(36%할인)         844800    24     4
           퓨어레드큐민(분말)_3박스(46%할인)          316000    12     4
           퓨어레드큐민(분말)_6박스(53%할인)          834000    36     6
2025-04-15 선택: 파이토레드큐민(정제)_12박스(45%할인)    363000    12     1
           선택: 파이토레드큐민(정제)_3박스(25%할인)     247500     6     2
           선택: 파이토레드큐민(정제)_6박스(36%할인)     422400    12     2
           선택: 퓨어레드큐민(분말)_3박스(46%할인)      474000    18     6
           선택: 퓨어레드큐민(분말)_6박스(53%할인)      139000     6     1
           퓨어레드큐민(분말)_12박스(55%할인)         267000    12     1
2025-04-16 선택: 파이토레드큐민(정제)_12박스(45%할인)    363000    12     1
           선택: 파이토레드큐민(정제)_3박스(25%할인)     495000    12     4
           선택: 파이토레드큐민(정제)_6박스(36%할인)     422400    12     2
           선택: 퓨어레드큐민(분말)_12박스(55%할인)     267000    12     1
           선택: 퓨어레드큐민(분말)_3박스(46%할인)      316000    12     4
           선택: 퓨어레드큐민(분말)_6박스(53%할인)      556000    24     4
2025-04-17 선택: 파이토레드큐민(정제)_3박스(25%할인)     247500     6     2
           선택: 파이토레드큐민(정제)_6박스(36%할인)     422400    12     2
           선택: 퓨어레드큐민(분말)_12박스(55%할인)     267000    12     1
           선택: 퓨어레드큐민(분말)_3박스(46%할인)      316000    12     4
           선택: 퓨어레드큐민(분말)_6박스(53%할인)      278000    12     2
2025-04-18 선택: 파이토레드큐민(정제)_12박스(45%할인)    726000    24     2
           선택: 파이토레드큐민(정제)_3박스(25%할인)     247500     6     2
           선택: 퓨어레드큐민(분말)_3박스(46%할인)      395000    15     5
           선택: 퓨어레드큐민(분말)_6박스(53%할인)      278000    12     2
           파이토레드큐민(정제)_12박스(45%할인)        363000    12     1
2025-04-19 선택: 파이토레드큐민(정제)_3박스(25%할인)     123750     3     1
           선택: 퓨어레드큐민(분말)_12박스(55%할인)     534000    24     2
           선택: 퓨어레드큐민(분말)_3박스(46%할인)      316000    12     4
           퓨어레드큐민(분말)_3박스(46%할인)           79000     3     1
2025-04-20 선택: 파이토레드큐민(정제)_3박스(25%할인)     742500    18     6
           선택: 파이토레드큐민(정제)_6박스(36%할인)     633600    18     3
           선택: 퓨어레드큐민(분말)_3박스(46%할인)      237000     9     3
           선택: 퓨어레드큐민(분말)_6박스(53%할인)      278000    12     2
           퓨어레드큐민(분말)_12박스(55%할인)         267000    12     1
           퓨어레드큐민(분말)_6박스(53%할인)          139000     6     1
2025-04-21 파이토레드큐민(정제)_3박스(25%할인)         123750     3     1
           퓨어레드큐민(분말)_3박스(46%할인)           79000     3     1
총합                                      16806500   573   108

In [16]:
df['제형'] = df['옵션명'].apply(lambda x: x.split('_')[0].split('(')[1])
df['제형'] = df['제형'].apply(lambda x: x[:2])
temp_df1 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '제형'], values=['매출', '박스수량'], aggfunc='sum', fill_value=0, margins=True, margins_name='총합'))
temp_df2 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '제형'], values=['결제 번호'], aggfunc='count', fill_value=0, margins=True, margins_name='총합'))
temp_df2.columns = ['주문수량']

result = pd.merge(temp_df1, temp_df2, left_index=True, right_index=True, how='inner')
result.to_excel(f'/Users/seanyu_mac/Downloads/샤인약사님_매출_{datetime.now()}.xlsx')
result

매출  박스수량  주문수량
결제 완료일     제형                      
2025-04-14 분말   1804000    69    15
           정제   3577200   108    15
2025-04-15 분말    880000    36     8
           정제   1032900    30     5
2025-04-16 분말   1139000    48     9
           정제   1280400    36     7
2025-04-17 분말    861000    36     7
           정제    669900    18     4
2025-04-18 분말    673000    27     7
           정제   1336500    42     5
2025-04-19 분말    929000    39     7
           정제    123750     3     1
2025-04-20 분말    921000    39     7
           정제   1376100    36     9
2025-04-21 분말     79000     3     1
           정제    123750     3     1
총합             16806500   573   108